In [1]:
import pyaspeller

In [2]:
speller = pyaspeller.create_speller()

TypeError: create_speller() missing 1 required positional argument: 'args'

In [3]:
import requests

In [28]:
params = {'text': 'какого цвеета ты сегодя?', 'lang': 'ru,en'}
res = requests.get('http://speller.yandex.net/services/spellservice.json/checkText', params=params)

In [32]:
res.json()

[{'code': 1,
  'col': 7,
  'len': 6,
  'pos': 7,
  'row': 0,
  's': ['цвета'],
  'word': 'цвеета'},
 {'code': 1,
  'col': 17,
  'len': 6,
  'pos': 17,
  'row': 0,
  's': ['сегодня'],
  'word': 'сегодя'}]

In [33]:
t = 'какого цвеета ты сегодя?'

In [35]:
t[7:7+6]

'цвеета'

In [ ]:
res.

In [ ]:
import requests


class Speller(object):
    service = 'http://speller.yandex.net/services/spellservice.json/checkText'

    def __init__(self, text, options=None, lang=None, format_text=None):
        self.text = text
        self.options = options
        self.lang = lang
        self.format_text = format_text
        self._answer = None

    def check(self):
        data = {'text': self.text}
        if self.options:
            data['options'] = self.options
        if self.lang:
            data['lang'] = self.lang
        if self.format_text:
            data['format'] = self.format_text
        answer = requests.post(url=self.service, data=data).json()
        return answer

    @property
    def answer(self):
        if self._answer is None:
            self._answer = self.check()
        return self._answer

    @property
    def correct(self):
        return not self.answer

    @property
    def spellsafe(self):
        raise NotImplementedError("Subclasses should implement this!")


class Word(Speller):

    @property
    def variants(self):
        if self.correct:
            return
        return self.answer[0]['s']

    @property
    def spellsafe(self):
        if self.correct:
            return
        return self.variants[0]


class Text(Speller):

    @property
    def spellsafe(self):
        changes = {el['word']: el['s'][0] for el in self.answer}
        result = self.text
        for wrong, fixed in changes.iteritems():
            result = result.replace(wrong, fixed)
        return result

    @property
    def errors(self):
        return [el['word'] for el in self.answer]

In [4]:
import pandas as pd

In [7]:
len("""
"408906692374446080";"1386325927";"pleease_shut_up";"@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)";"1";"0";"0";"0";"7569";"62";"61";"0"
""".split(';'))

12

In [26]:
data_pos = pd.read_csv('/media/data/nlp/sentiment/rus-mokoron/positive.csv',
                   names=['id', 'date', 'user', 'text', 'sentiment'], index_col=0,
                   sep=';', usecols=[0, 1, 2, 3, 4])
data_neg = pd.read_csv('/media/data/nlp/sentiment/rus-mokoron/negative.csv',
                   names=['id', 'date', 'user', 'text', 'sentiment'], index_col=0,
                   sep=';', usecols=[0, 1, 2, 3, 4])

data = pd.concat([data_pos, data_neg])

In [27]:
data.sample(5)

,date,user,text,sentiment
id,,,,
410071275969576960,1386603585,Pdb_ndv,пиздец на улице холодрыга:( \nоколел нахер:((,-1
411120679615209472,1386853782,AnyaChepkasova,с кем из мальчиков в классе хоршоо общаешься? ...,1
409193229830328320,1386394243,detocasal,"Предложение почитателям котов — постите, плз, ...",1
411012821867315200,1386828067,KatharineFlagg,"супер ,приезжай скорее ,жду выходных ))\nаааа ...",1
413979682376548352,1387535422,kt1345,RT @vsyo: Китай выбрасывает американскую кукур...,-1


In [23]:
data.shape

(114911, 4)

In [24]:
checked = []

In [ ]:
for line in data.iterrows():
    res = 